In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import math
import pickle
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import balanced_accuracy_score, roc_auc_score, \
                            classification_report, log_loss

from sklearn.linear_model import LogisticRegression

from multi_agent_simulator import BiddingAgent, BiddingEnvironment, BidStrategy

import matplotlib as mpl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks")
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
validation = pd.read_hdf('preprocessed.h5', 'validation')

In [3]:
# constant bidding
environment = BiddingEnvironment(validation.copy())

In [5]:
# example grid search const biddidng strategy
results = []
for const_bid in tqdm(np.arange(200,300,5)):

    agent = BiddingAgent(6250*1000, environment)
    bids = BidStrategy.const_bidding(const_bid, environment.lenght)

    agent.simulate(bids)
    results.append(agent.statistics())
    
results = pd.DataFrame(results)
results.sort_values('CTR', ascending=False).head(5)

100%|██████████| 20/20 [01:31<00:00,  4.00s/it]


,CTR,aCPC,aCPM,budget_left,clicks,impressions,lost,spend
11,0.000651,115.740741,0,0,54,82956,4397,6250000
2,0.000649,107.758621,0,0,58,89317,14386,6250000
10,0.000647,115.740741,0,0,54,83412,5102,6250000
12,0.000643,117.924528,0,0,53,82483,3702,6250000
4,0.000641,111.607143,0,0,56,87320,11174,6250000


In [6]:
# linear bidding based on LR pCTR

# load the trained LR
lr = pickle.load(open( "lr_model", "rb" ))

# user LR to predict pCTR
pCTR_lr = lr.predict_proba(validation.drop(columns=['payprice', 'click']))[:, 1]

# get baseline avgCTR
no_click, click = np.bincount(validation['click'].values)
avgCTR = click / (no_click + click)

# init agent
agent = BiddingAgent(6250*1000, environment)

# get bids provided given strategy
bids = BidStrategy.linear_bidding(pCTR_lr, avgCTR, 3.52)

# simulate the of the agent in the environment
agent.simulate(bids)

# get the perforamnce
agent.statistics()

{'CTR': 0.0012896971375648094,
 'aCPM': 0,
 'aCPC': 41.52029530201342,
 'spend': 6186524,
 'impressions': 115531,
 'clicks': 149,
 'lost': 188394,
 'budget_left': 63476}

In [7]:
# add agent to evironment without simulating it

bids_random1 = BidStrategy.random_bidding(300, 400, environment.lenght)
bids_random2 = BidStrategy.random_bidding(1000, 1500, environment.lenght)

# register bids to the environment
environment.register_bid(bids_random1)
environment.register_bid(bids_random2)

In [4]:
environment.get_bids()

array([ 23,  75,  65, ..., 108,  50,  40])

In [13]:
# linear bidding based on LR pCTR in the new envrionment

# init agent
agent = BiddingAgent(6250*1000, environment)

# get bids provided given strategy
bids = BidStrategy.linear_bidding(pCTR_lr, avgCTR, 3.52)

# simulate the of the agent in the environment
agent.simulate(bids)

# get the perforamnce
agent.statistics()

{'CTR': 0.020309184601394362,
 'aCPM': 0,
 'aCPC': 60.105402985074626,
 'spend': 4027062,
 'impressions': 3299,
 'clicks': 67,
 'lost': 300626,
 'budget_left': 2222938}